<div align="center">

### Group 01    
    
### Group Members:

| Name           | BITSID         | Weightage |
|:---------------|:---------------|:----------|
| Ajay Saxena    | 2021sc04160    |**100%**   |
| MOVVA MANASWI  | 2021sc04180    |**100%**   |
| PRARITA ARORA  | 2021sc04049    |**100%**   |
| ADITI          | 2021sc04050    |**100%**   |

</div>


#### Question no. 01. Read the data in yellow_tripdata_2020-06.csv file into a dataframe created in spark. 

In [19]:
data_path = "user/datat/input/yellow_tripdata_2020-06.csv"

df = spark.read.csv(data_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame to ensure it's loaded correctly
df.show(5)


+--------------------+---------------------+---------------+-------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|    01-06-2020 00:31|     01-06-2020 00:49|              1|          3.6|         140|          68|           1|       15.5|  3.0|    0.5|       4.0|         0.0|                  0.3|        23.3|
|    01-06-2020 00:42|     01-06-2020 01:04|              1|          5.6|          79|         226|           1|       19.5|  3.0|    0.5|       2.0|         0.0|                  0.3|        25.3|
|    

### Step 5: Explore the Data
Now we can use the PySpark DataFrame API to work with our data

In [20]:
#print the schema
df.printSchema()

root
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [21]:
#count the number of rows
df.count()

549760

 #### Question-2. Count the number of taxi trips for each hour 

In [27]:
# Convert pickup datetime string to timestamp
from pyspark.sql.functions import from_unixtime, unix_timestamp

df = df.withColumn("pickup_datetime", from_unixtime(unix_timestamp("tpep_pickup_datetime", 'dd-MM-yyyy HH:mm')).cast("timestamp"))


In [28]:
# To count the number of taxi trips for each hour, we need to extract the hour from the tpep_pickup_datetime column and then group by this hour. Use the hour function to extract the hour from the tpep_pickup_datetime column, and then group by this hour and count.
hourly_counts = df.groupBy(hour("pickup_datetime").alias("Hour")).agg(count("*").alias("Number of Trips")).orderBy("Hour")

# Show the hourly counts
hourly_counts.show(24)

+----+---------------+
|Hour|Number of Trips|
+----+---------------+
|   0|           8122|
|   1|           6643|
|   2|           5111|
|   3|           5124|
|   4|           7136|
|   5|           6955|
|   6|          14907|
|   7|          19957|
|   8|          24824|
|   9|          28408|
|  10|          31948|
|  11|          35190|
|  12|          38083|
|  13|          39475|
|  14|          40525|
|  15|          40971|
|  16|          38627|
|  17|          38225|
|  18|          34181|
|  19|          26477|
|  20|          18518|
|  21|          15020|
|  22|          13238|
|  23|          12095|
+----+---------------+



### Question-2.2.  Create a table view of the data frame created in step 1 above and write SparkSQL queries to find out the following:

 #### Question no.03. Average fare amount collected by hour of the day

In [29]:
from pyspark.sql.functions import from_unixtime, unix_timestamp, hour

# Convert the string column to a timestamp column
df_timestamp = df.withColumn("pickup_timestamp", from_unixtime(unix_timestamp("tpep_pickup_datetime", "dd-MM-yyyy HH:mm")))

# Extract the hour from the new timestamp column
df_with_hour = df_timestamp.withColumn("hour_of_day", hour("pickup_timestamp"))

# Create a new temp view with the extracted hour
df_with_hour.createOrReplaceTempView("taxi_data_with_hour")

# Now compute the average fare amount by hour of the day using SparkSQL
avg_fare_by_hour = spark.sql("""
    SELECT 
        hour_of_day, 
        AVG(fare_amount) as average_fare
    FROM 
        taxi_data_with_hour
    WHERE
        hour_of_day IS NOT NULL
    GROUP BY 
        hour_of_day
    ORDER BY 
        hour_of_day
""")

avg_fare_by_hour.show(24)  # to display averages for all 24 hours


+-----------+------------------+
|hour_of_day|      average_fare|
+-----------+------------------+
|          0|18.880695641467593|
|          1|27.534966129760434|
|          2| 30.12912737233388|
|          3|  35.2012607338016|
|          4|40.932777466367995|
|          5|20.005923795830288|
|          6| 11.69149459985242|
|          7|11.342811043744058|
|          8|11.184160087012579|
|          9|11.276688256829068|
|         10|11.909430324276952|
|         11|11.991532821824373|
|         12|11.864507260457405|
|         13| 11.58097226092462|
|         14|12.048847378161607|
|         15|12.755428717873595|
|         16|12.926983198280979|
|         17|13.052417266187044|
|         18|12.484358269213894|
|         19|12.241942818295133|
|         20| 13.67892590992549|
|         21| 14.87007723035953|
|         22|16.231789545248535|
|         23|17.937012815212864|
+-----------+------------------+



#### Question-4. Average fare amount compared to the average trip distance.
We want to calculate the average fare amount and average trip distance, and then determine the ratio of the average fare to the average distance for each hour of the day.

In [30]:
# Compute the average fare amount and average trip distance by hour of the day using SparkSQL
avg_fare_and_distance_by_hour = spark.sql("""
    SELECT 
        hour_of_day, 
        AVG(fare_amount) as average_fare,
        AVG(trip_distance) as average_distance,
        CASE 
            WHEN AVG(trip_distance) != 0 THEN AVG(fare_amount) / AVG(trip_distance)
            ELSE NULL
        END as fare_to_distance_ratio
    FROM 
        taxi_data_with_hour
    WHERE
        hour_of_day IS NOT NULL
    GROUP BY 
        hour_of_day
    ORDER BY 
        hour_of_day
""")

avg_fare_and_distance_by_hour.show(24)  # to display averages for all 24 hours


+-----------+------------------+------------------+----------------------+
|hour_of_day|      average_fare|  average_distance|fare_to_distance_ratio|
+-----------+------------------+------------------+----------------------+
|          0|18.880695641467593|5.3101957645899995|      3.55555547826877|
|          1|27.534966129760434| 6.998961312659949|     3.934150354561082|
|          2| 30.12912737233388| 7.810856975151618|    3.8573395298598507|
|          3|  35.2012607338016| 28.23991608118659|    1.2465072712185803|
|          4|40.932777466367995|11.320184977578476|    3.6159106540610617|
|          5|20.005923795830288| 5.961084112149523|       3.3560881576986|
|          6| 11.69149459985242| 3.082606829006503|     3.792729740892868|
|          7|11.342811043744058|3.5050804229092534|    3.2361057879320803|
|          8|11.184160087012579|2.5761774895262635|     4.341377926203854|
|          9|11.276688256829068|2.5608205435088713|     4.403544904938007|
|         10|11.909430324

#### Question no.5. Average fare amount and average trip distance by day of the week

To calculate the average fare amount and average trip distance by day of the week, we would need to extract the day of the week from the tpep_pickup_datetime (or pickup_timestamp we've converted it to a timestamp format).

Convert tpep_pickup_datetime to a timestamp format.<br>
Extract the day of the week from the timestamp.<br>
Group by the day of the week to compute the average fare and trip distance.

In [32]:
from pyspark.sql.functions import dayofweek, date_format

# Convert the string column to a timestamp column 
df_timestamp = df.withColumn("pickup_timestamp", from_unixtime(unix_timestamp("tpep_pickup_datetime", "dd-MM-yyyy HH:mm")))

# Extract the day of the week from the new timestamp column
# we can use date_format to get the name of the day, or dayofweek to get the numeric representation (1 = Sunday, 2 = Monday, etc.)
df_with_day = df_timestamp.withColumn("day_of_week", date_format("pickup_timestamp", "EEEE"))

# Create a new temp view with the day of the week
df_with_day.createOrReplaceTempView("taxi_data_with_day")

# Now compute the average fare amount and average trip distance by day of the week using SparkSQL
avg_fare_and_distance_by_day = spark.sql("""
    SELECT 
        day_of_week, 
        AVG(fare_amount) as average_fare,
        AVG(trip_distance) as average_distance
    FROM 
        taxi_data_with_day
    GROUP BY 
        day_of_week
    ORDER BY 
        CASE day_of_week 
            WHEN 'Sunday' THEN 1
            WHEN 'Monday' THEN 2
            WHEN 'Tuesday' THEN 3
            WHEN 'Wednesday' THEN 4
            WHEN 'Thursday' THEN 5
            WHEN 'Friday' THEN 6
            WHEN 'Saturday' THEN 7
        END
""")

avg_fare_and_distance_by_day.show(7)  # to display averages for all days of the week


+-----------+------------------+------------------+
|day_of_week|      average_fare|  average_distance|
+-----------+------------------+------------------+
|     Sunday|14.686292601998977| 3.910443634278223|
|     Monday|13.409943486081076| 4.262469750235043|
|    Tuesday| 13.44987940367479| 3.202572362689536|
|  Wednesday|13.279588696533812|6.0476238081182645|
|   Thursday|13.404982738924733|3.9995484502692804|
|     Friday|13.702733958318719| 3.778188952327255|
|   Saturday|13.880926298071149|3.6022067638824393|
+-----------+------------------+------------------+



#### Q 6 In the month of June 2020, find the zone which had maximum number of pick ups.

To solve this, we need the PULocationID from the yellow_tripdata_2020-06.csv file to determine the pick-up locations and corresponding zone from  taxi+_zone_lookup.csv data which need to be read from HDFS into another dataframe.
Join this new dataframe with the previously created dataframe on the PULocationID column ( PULocationID in yellow_tripdata_2020-06.csv matches LocationID in taxi+_zone_lookup.csv).<br>
Group by the zone column and count the occurrences for each zone.<br>
Order the result in descending order and pick the first row.

In [33]:
# Step 1: Read the taxi+_zone_lookup.csv data
zone_lookup_path = "user/datat/input/taxi+_zone_lookup.csv"
zone_df = spark.read.csv(zone_lookup_path, header=True, inferSchema=True)

# Check the schema to confirm the structure
zone_df.printSchema()

# Step 2: Join the two dataframes
joined_df = df.join(zone_df, df.PULocationID == zone_df.LocationID, how='left')

# Step 3: Group by zone and count
zone_counts = joined_df.groupBy("Zone").count()

# Step 4: Order by count in descending order and display the first row
top_zone = zone_counts.orderBy("count", ascending=False).first()

print(f"The zone with the maximum number of pickups in June 2020 is: {top_zone['Zone']} with {top_zone['count']} pickups.")


root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

The zone with the maximum number of pickups in June 2020 is: Upper East Side North with 23098 pickups.


#### Q.07. In the month of June 2020, find the zone which had maximum number of drops. 
To find the zone with the maximum number of drops for June 2020, we would follow a similar approach to the previous answer. However, this time we would be focusing on the DOLocationID column (DOLocationID represents the drop-off location in the yellow_tripdata_2020-06.csv dataset).

In [34]:
# The zone lookup data is already loaded in the previous step as zone_df

# Join the dataframe using DOLocationID for drop-offs
joined_drop_df = df.join(zone_df, df.DOLocationID == zone_df.LocationID, how='left')

# Group by Zone and count for drop-offs
drop_zone_counts = joined_drop_df.groupBy("Zone").count()

# Order by count in descending order and get the top zone
top_drop_zone = drop_zone_counts.orderBy("count", ascending=False).first()

print(f"The zone with the maximum number of drop-offs in June 2020 is: {top_drop_zone['Zone']} with {top_drop_zone['count']} drop-offs.")


The zone with the maximum number of drop-offs in June 2020 is: Upper East Side North with 22254 drop-offs.


#### Q.08.Average no of passengers by hour of the day
To compute the average number of passengers by hour of the day, we'll use the df_with_hour dataframe from our previously provided code, which already has the hour_of_day extracted from the pickup_timestamp.

In [35]:
from pyspark.sql import functions as F

# Group by hour_of_day and compute average passengers
avg_passengers_per_hour = df_with_hour.groupBy("hour_of_day") \
                                     .agg(F.avg("passenger_count").alias("avg_passenger_count"))

# Order the results by hour_of_day for better readability
ordered_avg_passengers = avg_passengers_per_hour.orderBy("hour_of_day")

# Display the results
ordered_avg_passengers.show(24)


+-----------+-------------------+
|hour_of_day|avg_passenger_count|
+-----------+-------------------+
|          0| 1.3300081766148815|
|          1| 1.2963541666666667|
|          2|  1.308466051969824|
|          3| 1.3093270365997638|
|          4| 1.2861205915813425|
|          5| 1.3718697829716193|
|          6| 1.3303137428192664|
|          7| 1.3431017976810977|
|          8|  1.359296915388592|
|          9| 1.3521955975550306|
|         10|  1.361447777998543|
|         11| 1.3555843529624496|
|         12| 1.3567879870492352|
|         13| 1.3521634939012896|
|         14| 1.3638007863695938|
|         15| 1.3572915863345416|
|         16|   1.35842077865147|
|         17| 1.3650200560470356|
|         18|  1.376865328634901|
|         19| 1.3628078030060762|
|         20| 1.3558048103607772|
|         21| 1.3686376434914447|
|         22| 1.3518518518518519|
|         23| 1.3064166486017776|
+-----------+-------------------+



#### Q.09. Total number of payments made by different type for the month.
To determine the total number of payments made by different payment types for the month, we would group by month and payment_type column and count the number of occurrences for each type.

In [37]:
from pyspark.sql import functions as F

# Convert the string column to a timestamp column
df_timestamp = df.withColumn("pickup_timestamp", F.from_unixtime(F.unix_timestamp("tpep_pickup_datetime", "dd-MM-yyyy HH:mm")))

# Extract the month from the timestamp column
df_month = df_timestamp.withColumn("Month", F.month("pickup_timestamp"))

# Generate the PaymentDescription column
payment_description = F.when(df_month["payment_type"] == '1', 'Credit card') \
                       .when(df_month["payment_type"] == '2', 'Cash') \
                       .when(df_month["payment_type"] == '3', 'No charge') \
                       .when(df_month["payment_type"] == '4', 'Dispute') \
                       .when(df_month["payment_type"] == '5', 'Unknown') \
                       .when(df_month["payment_type"] == '6', 'Voided trip') \
                       .otherwise('Other').alias('PaymentDescription')

# Group by month and payment_type, then aggregate
agg_df = df_month.groupBy("Month", "payment_type") \
                 .agg(F.count("*").alias("TotalCount"))

# Add the PaymentDescription column to the result
result_df = agg_df.withColumn("PaymentDescription", payment_description)

# Order the result by Month and payment_type
ordered_result = result_df.orderBy("Month", "payment_type")

ordered_result.show()


+-----+------------+----------+------------------+
|Month|payment_type|TotalCount|PaymentDescription|
+-----+------------+----------+------------------+
|    1|           2|         3|              Cash|
|    5|           1|         1|       Credit card|
|    5|           2|         3|              Cash|
|    6|        null|     50717|             Other|
|    6|           1|    322565|       Credit card|
|    6|           2|    168937|              Cash|
|    6|           3|      5245|         No charge|
|    6|           4|      2275|           Dispute|
|    6|           5|        12|           Unknown|
|    7|           1|         2|       Credit card|
+-----+------------+----------+------------------+

